In [3]:
import os
import requests

def download_github_folder(owner, repo, path, branch="main", output_dir="lyrics_telecharges"):
    # URL de l'API GitHub pour lister le contenu du dossier
    api_url = f"https://api.github.com/repos/{owner}/{repo}/contents/{path}?ref={branch}"

    print(f"Récupération de la liste des fichiers depuis : {api_url} ...")
    response = requests.get(api_url)

    if response.status_code != 200:
        print(f"❌ Erreur lors de l'accès au dépôt (Code {response.status_code}). Vérifie l'URL.")
        return

    files_list = response.json()

    # Création du dossier de destination s'il n'existe pas
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Dossier créé : {output_dir}")

    count = 0

    # Boucle sur chaque élément trouvé
    for file_info in files_list:
        # On vérifie :
        # 1. C'est un fichier (pas un dossier)
        # 2. Ça finit par .txt
        # 3. La taille est strictement supérieure à 0
        if (file_info['type'] == 'file' and
            file_info['name'].endswith('.txt') and
            file_info['size'] > 0):

            download_url = file_info['download_url']
            file_name = file_info['name']

            # Téléchargement du contenu brut
            r = requests.get(download_url)
            if r.status_code == 200:
                file_path = os.path.join(output_dir, file_name)
                with open(file_path, 'wb') as f:
                    f.write(r.content)
                count += 1
                # Optionnel : afficher un point pour chaque fichier pour ne pas spammer la console
                if count % 10 == 0:
                    print(f"{count} fichiers téléchargés...", end="\r")
            else:
                print(f"Échec du téléchargement pour {file_name}")

    print(f"\n✅ Terminé ! {count} fichiers .txt (non vides) ont été téléchargés dans '{output_dir}'.")

# --- Configuration ---
# Basé sur ton lien : https://github.com/ras0k/whisper-rap-queb/tree/main/top999/genius_scrape
user = "ras0k"
repository = "whisper-rap-queb"
folder_path = "top999/genius_scrape"
branch_name = "main"

# Exécution
download_github_folder(user, repository, folder_path, branch_name)

Récupération de la liste des fichiers depuis : https://api.github.com/repos/ras0k/whisper-rap-queb/contents/top999/genius_scrape?ref=main ...
Dossier créé : lyrics_telecharges

✅ Terminé ! 328 fichiers .txt (non vides) ont été téléchargés dans 'lyrics_telecharges'.


In [4]:
import os

def chercher_texte(racine, terme_recherche):
    print(f"Recherche de '{terme_recherche}' dans : {racine}...\n")
    fichiers_trouves = []

    # Parcours de l'arborescence de fichiers
    for root, dirs, files in os.walk(racine):
        for filename in files:
            chemin_complet = os.path.join(root, filename)

            try:
                # On essaie d'ouvrir le fichier en lecture seule
                # 'errors="ignore"' permet de ne pas planter si on tombe sur un fichier binaire (image, etc.)
                with open(chemin_complet, 'r', encoding='utf-8', errors='ignore') as f:
                    contenu = f.read()
                    if terme_recherche in contenu:
                        print(f"✅ Trouvé dans : {chemin_complet}")
                        fichiers_trouves.append(chemin_complet)
            except Exception as e:
                # En cas d'erreur (permission, etc.), on ignore le fichier
                continue

    if not fichiers_trouves:
        print("\nAucun fichier contenant ce terme n'a été trouvé.")
    else:
        print(f"\nTerminé. {len(fichiers_trouves)} fichier(s) trouvé(s).")

# --- Configuration ---
# Par défaut, Colab travaille dans "/content".
# Si tu veux chercher dans ton Drive, monte-le d'abord et change le chemin vers "/content/drive/MyDrive"
dossier_de_depart = "/content"
mot_a_chercher = "follow"

# Lancer la recherche
chercher_texte(dossier_de_depart, mot_a_chercher)

Recherche de 'follow' dans : /content...

✅ Trouvé dans : /content/lyrics_telecharges/041-Corops-dis-moi.txt
✅ Trouvé dans : /content/lyrics_telecharges/121-Enima-reussir.txt
✅ Trouvé dans : /content/lyrics_telecharges/283-White-b-solo.txt
✅ Trouvé dans : /content/lyrics_telecharges/181-Koriass-lait-de-chevre.txt
✅ Trouvé dans : /content/lyrics_telecharges/113-Koriass-zombies.txt
✅ Trouvé dans : /content/lyrics_telecharges/053-Dead-obies-where-they.txt

Terminé. 6 fichier(s) trouvé(s).
